In [11]:
## Importing Libraries from Python ##  -  Import all libraries to be used here.
import pandas as pd
import numpy as np
import spacy
import string

In [12]:
## Read the dataset using delimitter '\t' ##
## Also print the dataset ##
DataSet = pd.read_csv('amazon_alexa_TEST.tsv',sep = '\t')
print(DataSet)
DataSet.shape

      rating       date         variation  \
0          5  31-Jul-18  Charcoal Fabric    
1          5  31-Jul-18  Charcoal Fabric    
2          4  31-Jul-18    Walnut Finish    
3          5  31-Jul-18  Charcoal Fabric    
4          5  31-Jul-18  Charcoal Fabric    
...      ...        ...               ...   
3145       5  30-Jul-18        Black  Dot   
3146       5  30-Jul-18        Black  Dot   
3147       5  30-Jul-18        Black  Dot   
3148       5  30-Jul-18        White  Dot   
3149       4  29-Jul-18        Black  Dot   

                                       verified_reviews  feedback  
0                                         Love my Echo!         1  
1                                             Loved it!         1  
2     Sometimes while playing a game, you can answer...         1  
3     I have had a lot of fun with this thing. My 4 ...         1  
4                                                 Music         1  
...                                                

(3150, 5)

In [7]:
## Describing the dataset using describe() ##
## This provides us information about the dataset ##
DataSet.describe()

,rating,feedback
count,3150.000000,3150.000000
mean,4.463175,0.918413
std,1.068506,0.273778
min,1.000000,0.000000
25%,4.000000,1.000000
50%,5.000000,1.000000
75%,5.000000,1.000000
max,5.000000,1.000000


In [8]:
## Print the first 5 rows of the dataset for viewing the details of each row and column ##
DataSet.head()

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [10]:
## Word Tokenization

nlp = spacy.load("en_core_web_sm")
for row in range(len(DataSet)):
    reviews = DataSet.loc[row, 'verified_reviews']
    reviews = reviews.translate(str.maketrans('', '', string.punctuation))
    doc = nlp(reviews)
    for token in doc:
        print(token.text)

NameError: name 'string' is not defined